# Построение НКА по регулярному выражению

In [1029]:
def star(graph: dict,
         symbol: str,
         cur_state: int):
    graph[f"A{cur_state}"] = [("EMPTY", f"A{cur_state + 1}")]
    cur_state += 1

    if len(symbol) == 1:
        graph[f"A{cur_state}"] = [(symbol, f"A{cur_state}")]
        cur_state += 1
    elif len(symbol) > 1:
        start_state = cur_state
        for i in symbol:
            graph[f"A{cur_state}"] = [(i, f"A{cur_state + 1}")]
            cur_state += 1
        
        graph[f"A{cur_state - 1}"].append((symbol[0], f"A{start_state}"))
    
    graph[f"A{cur_state - 1}"].append(("EMPTY", f"A{cur_state}"))

    return graph, cur_state


def question(graph: dict,
             symbol: str,
             cur_state: int):
    if len(symbol) == 1:
        graph[f"A{cur_state}"] = [(symbol, f"A{cur_state + 1}")]
        cur_state += 1

        graph[f"A{cur_state - 1}"].append(("EMPTY", f"A{cur_state}"))
    elif len(symbol) > 1:
        start_state = cur_state

        for i in symbol:
            graph[f"A{cur_state}"] = [(i, f"A{cur_state + 1}")]
            cur_state += 1
        
        graph[f"A{start_state}"].append(("EMPTY", f"A{cur_state}"))

    return graph, cur_state


def plus(graph: dict,
         symbol: str,
         cur_state: int):
    if len(symbol) == 1:
        graph[f"A{cur_state}"] = [(symbol, f"A{cur_state + 1}")]
        cur_state += 1
        graph[f"A{cur_state}"] = [(symbol, f"A{cur_state}")]
        cur_state += 1
    elif len(symbol) > 1:
        start_state = cur_state
        for i in symbol:
            graph[f"A{cur_state}"] = [(i, f"A{cur_state + 1}")]
            cur_state += 1
        
        graph[f"A{cur_state - 1}"].append((symbol[0], f"A{start_state}"))
    
    graph[f"A{cur_state - 1}"].append(("EMPTY", f"A{cur_state}"))

    return graph, cur_state


def slash(graph: dict,
          symbol: str,
          cur_state: int):
    pass

In [1030]:
def create_graph(s: str):
    cur_state = 0
    start_state = f"A{cur_state}"
    graph = dict()

    for i in range(len(s)):
        if i + 1 < len(s) and s[i + 1] == '*':
            graph, cur_state = star(graph, s[i], cur_state)
        elif i + 1 < len(s) and s[i + 1] == '+':
            graph, cur_state = plus(graph, s[i], cur_state)
        elif i + 1 < len(s) and s[i + 1] == '?':
            graph, cur_state = question(graph, s[i], cur_state)
        elif i + 2 < len(s) and s[i + 1] == '|':
            graph, cur_state = slash(graph, s[i], s[i + 2], cur_state)
        elif s[i] in ('*', '+', '?', '|'):
            continue
        else:
            graph[f"A{cur_state}"] = [(s[i], f"A{cur_state + 1}")]
            cur_state += 1
    
    return graph, start_state, f"A{cur_state}"

# Избавление от е-дуг

In [1031]:
def check_depth(graph: dict,
                cur_state: str,
                depth: int = 0,
                states: list = list()
                ) -> list:
    if cur_state in graph.keys():
        if depth == 0:
            for transit in graph[cur_state]:
                if transit[1] not in states and transit[0] != "EMPTY":
                    states.append(transit[1])
                    states.extend(check_depth(graph, transit[1], depth + 1, states))
        else:
            for transit in graph[cur_state]:
                if transit[1] not in states and transit[0] == "EMPTY":
                    if transit[1] not in states:
                        states.append(transit[1])
                        states.extend(check_depth(graph, transit[1], depth + 1, states))

    states = list(set(states))
    
    return states

In [1032]:
check_depth(nka, "A1", depth=0, states=list())

['A3', 'A2', 'A4']

In [1033]:
def delete_missing_states(graph: dict) -> dict:
    res = dict()

    missing = list()
    for state in graph.keys():
        if len(graph[state]) == 0:
            missing.append(state)
    
    for state in missing:
        graph.pop(state)

    for state in graph.keys():
        res[state] = list()
        for transit in graph[state]:
            if transit[1] not in missing:
                res[state].append(transit)
    
    return res

In [1034]:
def lose_emptys(graph: dict,
                start_state: str,
                fin_state: str):
    states = dict()
    emptys = dict()
    for state in graph.keys():
        states[state] = list()
        emptys[state] = check_depth(graph, state, depth=0, states=list())

    for state in states.keys():
        for transit in graph[state]:
            if transit[0] != "EMPTY":
                for emp in emptys[state]:
                    states[state].append((transit[0], emp))
    
    states = delete_missing_states(states)

    return states, start_state, fin_state

# Преобразование НКА в ДКА

In [1035]:
def determinize(graph: dict,
                start_state: str,
                fin_state: str):
    finites = set()
    new_graph = dict()
    alphabet = set()

    for cur_state in graph.keys():
        for transit in graph[cur_state]:
            k = (cur_state, transit[0])
            alphabet.add(transit[0])
            if k not in new_graph.keys():
                new_graph[k] = [transit[1]]
            else:
                new_graph[k].append(transit[1])

    all_states = False
    while not all_states:
        old_keys = list(new_graph.keys())
        for cur_state in old_keys:
            k = str(' '.join(sorted(new_graph[cur_state])))
            for letter in alphabet:
                temp = list()
                for prev_state in new_graph[cur_state]:
                    prev_key = (prev_state, letter)
                    if prev_key in old_keys:
                        temp.extend(new_graph[prev_key])
                if len(temp) > 0:
                    tup = (k, letter)
                    new_graph[tup] = temp

        all_keys = [k for k, _ in list(new_graph.keys())]
        for cur_state in new_graph:
            new_state = ' '.join(sorted(new_graph[cur_state]))
            if new_state in all_keys:
                all_states = True
            elif new_state in finites:
                all_states = True

    for cur_state in new_graph.keys():
        if fin_state in new_graph[cur_state]:
            finites.add(' '.join(sorted(new_graph[cur_state])))
            
        new_graph[cur_state] = ' '.join(sorted(new_graph[cur_state]))
    
    old_keys = list(new_graph.keys())
    for cur_state in old_keys:
        if cur_state[0] not in new_graph.values() and cur_state[0] != start_state:
            new_graph.pop(cur_state)

    return new_graph, start_state, list(finites)

# Разбор регулярного выражения по конечному автомату

In [1036]:
def checkup(regexp: str,
            s: str) -> bool:
    # Построение НКА
    nka, start_state, fin_state = create_graph(regexp)
    # print(f"{nka}\n{start_state}\n{fin_state}", end="\n\n")

    # Удаление е-дуг
    nka, start_state, fin_state = lose_emptys(nka, start_state, fin_state)
    # print(f"{nka}\n{start_state}\n{fin_state}", end="\n\n")

    # Построение ДКА
    dka, start_state, fin_state = determinize(nka, start_state, fin_state)
    # print(f"{dka}\n{start_state}\n{fin_state}", end="\n\n")

    res = False
    cur_state = start_state
    for symbol in s:
        k = (cur_state, symbol)
        if k in dka.keys() and cur_state not in fin_state:
            cur_state = dka[k]
        elif k not in dka.keys():
            cur_state = start_state
            break
    if cur_state in fin_state:
        res = True
    else:
        res = False
    
    return res

In [1037]:
regexp = "he+n?lo*"
s = ["hello", "hola", "hel", "henlooo", "heeeeeeeeeeeeeelo", "hnelo", "heenllllllo", "helno"]
for i in s:
    print(checkup(regexp, i))

False
False
True
True
True
False
False
False


In [1038]:
regexp = "he+n?l*o"
s = ["hello", "hola", "hel", "henlooo", "heeeeeeeeeeeeeelo", "hnelo", "heenllllllo", "helno"]
for i in s:
    print(checkup(regexp, i))

True
False
False
False
True
False
True
False
